In [ ]:
##### Copyright 2020 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# We are using NitroML on Kubeflow: 

This notebook allows users to analyze NitroML benchmark results.

In [ ]:
# This notebook assumes you have followed the following steps to setup port-forwarding:

# Step 1: Configure your cluster with gcloud
# `gcloud container clusters get-credentials <cluster_name> --zone <cluster-zone> --project <project-id>

# Step 2: Get the port where the gRPC service is running on the cluster
# `kubectl get configmap metadata-grpc-configmap -o jsonpath={.data}`
# Use `METADATA_GRPC_SERVICE_PORT` in the next step. The default port used is 8080.

# Step 3: Port forwarding
# `kubectl port-forward deployment/metadata-grpc-deployment 9898:<METADATA_GRPC_SERVICE_PORT>`

# Troubleshooting
# If getting error related to Metadata (For examples, Transaction already open). Try restarting the metadata-grpc-service using:
# `kubectl rollout restart deployment metadata-grpc-deployment`  

In [ ]:
import sys, os
PROJECT_DIR=os.path.join(sys.path[0], '..')
%cd {PROJECT_DIR}

In [ ]:
from ml_metadata.proto import metadata_store_pb2
from ml_metadata.metadata_store import metadata_store
from nitroml.benchmark import results

## Connect to the ML Metadata (MLMD) database

First we need to connect to our MLMD database which stores the results of our
benchmark runs.

In [ ]:
connection_config = metadata_store_pb2.MetadataStoreClientConfig()

connection_config.host = 'localhost'
connection_config.port = 9898

store = metadata_store.MetadataStore(connection_config)

## Display benchmark results

Next we load and visualize `pd.DataFrame` containing our benchmark results.
These results contain contextual features such as the pipeline ID, and 
benchmark metrics as computed by the downstream Evaluators. If your
benchmark included an `EstimatorTrainer` component, its hyperparameters may also
display in the table below.

In [ ]:
#@markdown ### Choose how to aggregate metrics:
mean = False  #@param { type: "boolean" }
stdev = False  #@param { type: "boolean" }
min_and_max = False  #@param { type: "boolean" }

agg = []
if mean:
    agg.append("mean")
if stdev:
    agg.append("std")
if min_and_max:
    agg += ["min", "max"]

df = results.overview(store, metric_aggregators=agg)

In [ ]:
df.head()

### We can display an interactive table using qgrid

Please follow the latest instructions on downloading qqgrid package from here: https://github.com/quantopian/qgrid

In [ ]:
import qgrid
qgid_wdget = qgrid.show_grid(df, show_toolbar=True)
qgid_wdget